In [9]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV

In [2]:
data = pd.read_csv("C:/Users/paul8/Downloads/iris.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
data["species"] = data["species"] == "virginica"
X = data.drop(["species"], axis=1)
Y = data["species"]

In [5]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=2,
shuffle=True, stratify=Y)

In [6]:
xgb = XGBClassifier(objective='binary:logistic')

In [7]:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 1001)

In [27]:
params = {'n_estimators' : [(x+1)**2 for x in range(9)]}
random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=4, cv=skf.split(Xtrain,Ytrain), verbose=3, random_state=42)
random_search.fit(Xtrain, Ytrain)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000020E84117B30>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_...
                                           max_cat_threshold=None,
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=5, n_jobs=4,
                   param_distributions={'n_estimators': [1, 4, 9, 16, 25, 36,
                                                         49, 64, 81]},
                   random_state=42, scoring='roc_auc', verbose=3)

In [28]:
res = pd.DataFrame(random_search.cv_results_)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018198,0.003349,0.004725,0.002124,64,{'n_estimators': 64},1.0,1.0,0.964286,0.928571,0.750000,1.0,0.904762,1.0,1.00000,1.0,0.954762,0.075818,2
1,0.005672,0.003442,0.005183,0.002050,4,{'n_estimators': 4},1.0,1.0,0.982143,0.892857,0.857143,1.0,0.904762,1.0,1.00000,1.0,0.963690,0.053003,1
2,0.011819,0.007401,0.003623,0.005502,36,{'n_estimators': 36},1.0,1.0,0.964286,0.928571,0.750000,1.0,0.904762,1.0,1.00000,1.0,0.954762,0.075818,2
3,0.004058,0.003265,0.004505,0.003557,1,{'n_estimators': 1},1.0,1.0,0.982143,0.964286,0.857143,1.0,0.761905,1.0,0.97619,1.0,0.954167,0.076323,5
4,0.018095,0.003599,0.004603,0.002344,81,{'n_estimators': 81},1.0,1.0,0.964286,0.928571,0.750000,1.0,0.904762,1.0,1.00000,1.0,0.954762,0.075818,2


In [29]:
random_search.best_params_

{'n_estimators': 4}

In [30]:
model = random_search.best_estimator_
res_best = model.predict(Xtest)

In [31]:
xgb.fit(Xtrain,Ytrain)
res_default = xgb.predict(Xtest)

In [32]:
acc=sum(res_best==Ytest)/len(Ytest)
print(f"accuracy of tuned model = {round(acc*100,2)} %")

acc=sum(res_default==Ytest)/len(Ytest)
print(f"accuracy of default model = {round(acc*100,2)} %")

accuracy of tuned model = 97.78 %
accuracy of default model = 97.78 %


In [33]:
confusion_matrix = pd.crosstab(Ytest, res_best, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)
print("\n")
confusion_matrix = pd.crosstab(Ytest, res_default, rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

Predicted   0   1
Actual           
False      29   1
True        0  15


Predicted   0   1
Actual           
False      29   1
True        0  15
